In [1]:
#概要
"""
住宅の価格を特徴量から予測する

評価方法 RMSE
"""

'\n住宅の価格を特徴量から予測する\n\n評価方法 RMSE\n'

In [28]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")
print("data size \ntrain : {}\ntest : {}".format(len(train), len(test)))

data size 
train : 1460
test : 1459


In [29]:
#ベースラインとしてxgboost回帰を行う
#特徴量の作成
#trainを学習用とValidation用に分ける 1000 : 460

In [30]:
#数値データと名前データに分ける
#いくつかは数値だがカテゴリーデータとして扱う
columns = train.columns
val_col = []
cat_col = []
import math

for c in columns:
    data = train[c]
    Cats = ["MSSubClass", "MoSold", "YrSold"]
    if c in Cats:
        cat_col.append(c)
        continue
        
    for d in data:
        if type(d) == str:
            cat_col.append(c)
            break
        if math.isnan(d):
            continue
        else:
            val_col.append(c)
            break
            

print(cat_col, val_col)
print("val num :{} \ncat num :{}".format(len(val_col), len(cat_col)))
print("total num", len(val_col) + len(cat_col))
print("base num", len(columns))


['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition'] ['Id', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckS

In [79]:
#特徴ベクトルに変換する
#valのnanは０うめ
def pd2feats(data_pd):
    feats_pd = data_pd.loc[:,val_col].copy()
    feats_pd = feats_pd.fillna(0)
    
    for cn in cat_col:
        data = data_pd[cn].copy()
        data[data != data] = "NA"
        dummy = pd.get_dummies(data)
        feats_pd = pd.concat([feats_pd, dummy], axis=1)

    return feats_pd

train_pd = pd2feats(train)

In [96]:
#特徴ベクトルの作成
_train_pd = train_pd.copy()
del _train_pd["Id"]
del _train_pd["SalePrice"]
train_arr = np.array(_train_pd)
train_size = 1000
train_data = train_arr[:train_size].copy()
val_data = train_arr[train_size:].copy()
train_target = np.log(np.array(train_pd.loc[:,["SalePrice"]].copy()))
train_target_arr = train_target[:train_size].copy()
val_target_arr = train_target[train_size:].copy()


In [97]:
#xgboost
import xgboost as xgb
model = xgb.XGBRegressor(max_depth=4, n_estimators=250, 
                         min_child_weight=3,reg_alpha=0.05,
                        reg_lambda=0.05, sumsample=0.8)
model.fit(train_data, train_target_arr)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=3, missing=None, n_estimators=250,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0.05, reg_lambda=0.05, scale_pos_weight=1, seed=None,
       silent=True, subsample=1, sumsample=0.8)

In [106]:
#RMSE算出

def CalRMSE(model):
    train_predict = model.predict(train_data)
    val_predict = model.predict(val_data)
    train_rmse = np.sqrt(np.mean((train_predict - train_target_arr[:,0])**2))
    val_rmse = np.sqrt(np.mean((val_predict - val_target_arr[:,0])**2))
    print("Train RMSE :", train_rmse)
    print("Val RMSE :", val_rmse)

CalRMSE(model)




Train RMSE : 0.0335301449164
Val RMSE : 0.126619289858


{'Id': -0.021916719443431106, 'MSSubClass': -0.084284135126595189, 'LotFrontage': 0.33477085313976007, 'LotArea': 0.26384335387140562, 'OverallQual': 0.79098160058380496, 'OverallCond': -0.077855894048678015, 'YearBuilt': 0.52289733287949691, 'YearRemodAdd': 0.50710096711138619, 'MasVnrArea': 0.47261449900457386, 'BsmtFinSF1': 0.38641980624215339, 'BsmtFinSF2': -0.011378121450215144, 'BsmtUnfSF': 0.21447910554696892, 'TotalBsmtSF': 0.61358055155919544, '1stFlrSF': 0.6058521846919146, '2ndFlrSF': 0.31933380283206786, 'LowQualFinSF': -0.025606130000679544, 'GrLivArea': 0.70862447761265224, 'BsmtFullBath': 0.22712223313149427, 'BsmtHalfBath': -0.016844154297359009, 'FullBath': 0.56066376274844609, 'HalfBath': 0.28410767559478262, 'BedroomAbvGr': 0.16821315430073999, 'KitchenAbvGr': -0.13590737084214122, 'TotRmsAbvGrd': 0.53372315558202821, 'Fireplaces': 0.4669288367515278, 'GarageYrBlt': 0.46675365236333943, 'GarageCars': 0.64040919725835221, 'GarageArea': 0.62343143891836172, 'WoodDeckSF

In [108]:
a = np.array([1,2,3,4,5])
b = np.array([2,3,4,5,6]).reshape(-1,1)


array([[-1,  0,  1,  2,  3],
       [-2, -1,  0,  1,  2],
       [-3, -2, -1,  0,  1],
       [-4, -3, -2, -1,  0],
       [-5, -4, -3, -2, -1]])

In [107]:
#特に何も考えず線形回帰 or xgboost回帰により住宅価格の予測を行う

a = np.array([1,2,3,4,5])
np.var(a)*5 

10.0

In [108]:
np.sum((a - 3)**2)

10